In [1]:
%matplotlib notebook

In [2]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import requests
import json
from config import gkey
from config import omdb_api_key

In [3]:
csvpath09_19 = os.path.join("C:/Users/joeki/Desktop/Datamaniacs/COBRA-2009-2019.csv")
csvpath20_new = os.path.join("C:/Users/joeki/Desktop/Datamaniacs/COBRA-2020(NEW RMS 9-30 12-31).csv")
csvpath20_old = os.path.join("C:/Users/joeki/Desktop/Datamaniacs/COBRA-2020-OldRMS-09292020.csv")
csvpath21 = os.path.join("C:/Users/joeki/Desktop/Datamaniacs/COBRA-2021.csv")


In [4]:
#  csvpath09_19_df   does not like to merge
csvpath09_19_df = pd.read_csv(csvpath09_19, encoding="utf-8")
csvpath09_19_df.head()

csvpath20_new_df = pd.read_csv(csvpath20_new, encoding="utf-8")
csvpath20_new_df.head()

csvpath20_old_df = pd.read_csv(csvpath20_old, encoding="utf-8")
csvpath20_old_df.head()

csvpath21_df = pd.read_csv(csvpath21, encoding="utf-8")
csvpath21_df.head()

#### KEY ERROR on "offense_id"   "csvpath09_19_df" does not like to merge
# merge09_19_20new_df = pd.merge(csvpath09_19_df, csvpath20_new_df, how="left", on="offense_id")
# merge09_19_20new_df

# # KEY ERROR on "offense_id"     "csvpath09_19_df" does not like to merge
# merge0919_csvpath21_df_df = pd.merge(csvpath09_19_df, csvpath21_df, how="left", on="offense_id")
# merge0919_csvpath21_df_df.head()

#### KEY ERROR on "offense_id"       "csvpath09_19_df" does not like to merge
# merge09_19_2020_df = pd.merge(csvpath09_19_df, csvpath21_df, how="left", on="offense_id")
# merge09_19_2020_df

C:\Users\joeki\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,offense_id,rpt_date,occur_date,occur_time,poss_date,poss_time,beat,location,ibr_code,UC2_Literal,neighborhood,npu,lat,long
0,200006928,10/4/2020,7/8/2020,10:00,7/8/2020,15:00,103,1888 HOLLYWOOD RD NW,240,AUTO THEFT,Riverside,D,33.807140,-84.471539
1,200016928,10/4/2020,6/13/2020,13:00,6/13/2020,13:30,603,595 PIEDMONT AVE NE,23C,LARCENY-NON VEHICLE,Old Fourth Ward,M,33.770357,-84.381158
2,200201253,1/20/2021,1/20/2021,04:00,1/20/2021,04:30,110,2479 ABNER TERRACE NW,23G,LARCENY-FROM VEHICLE,Scotts Crossing,G,33.800999,-84.466556
3,201791305,10/5/2020,10/5/2020,16:00,10/5/2020,17:00,207,2143 PEACHTREE RD,23H,LARCENY-NON VEHICLE,NaN,E,33.812773,-84.391538
4,202101654,11/6/2020,11/5/2020,19:37,11/5/2020,19:38,305,177 MOURY AVE SE,13A,AGG ASSAULT,The Villages at Carver,Y,33.713540,-84.390301


In [5]:
# Merging the New and Old datasets 2020
merge2020_df = pd.merge(csvpath20_new_df, csvpath20_old_df, how="left", on="offense_id")
merge2020_df.head()
offense_ids = len(merge2020_df["offense_id"].unique())
print(f"There are {offense_ids} 'offense_ids'.")

There are 7245 'offense_ids'.


In [6]:
# Find Duplicates
duplicate_merge2020_df = merge2020_df.loc[merge2020_df.duplicated(subset=["offense_id"]), "offense_id"].unique()
duplicate_merge2020_df
print(f"These are the duplicate offense ids: {duplicate_merge2020_df}")

These are the duplicate offense ids: [203530800 203081476 203020020 202900817]


In [7]:
# Dropped 
clean_merge2020_df = merge2020_df[merge2020_df["offense_id"].isin(duplicate_merge2020_df)==False]
clean_merge2020_df.head()
clean_ids = len(clean_merge2020_df["offense_id"].unique())
print(f"There are {clean_ids} 'offense_ids' in the cleaned dataframe.")

There are 7241 'offense_ids' in the cleaned dataframe.


In [8]:
# Merge 2020df with 2021  ###  NOT CLEANED DATAFRAME   #######
new_merged202021_df = pd.merge(merge2020_df, csvpath21_df, how="left", on="offense_id")
new_merged202021_df.head()

,offense_id,rpt_date_x,occur_date_x,occur_time_x,poss_date_x,poss_time_x,beat_x,apt_office_prefix,apt_office_num,location_x,...,poss_date,poss_time,beat,location,ibr_code_y,UC2_Literal,neighborhood,npu,lat,long
0,203341346,12/31/2020,11/29/2020,19:21,11/29/2020,23:50,405,NaN,NaN,3050 M.L.K. JR DR SW,...,11/29/2020,23:50,405,3050 M.L.K. JR DR SW,13A,AGG ASSAULT,Harland Terrace,I,33.751060,-84.486163
1,203631200,12/31/2020,12/28/2020,17:21,12/28/2020,17:45,302,NaN,NaN,1186 IRA ST SW,...,12/28/2020,17:45,302,1186 IRA ST SW,220,BURGLARY,Pittsburgh,V,33.722696,-84.399763
2,203631285,12/31/2020,12/24/2020,14:00,12/24/2020,18:26,603,NaN,NaN,501 BOULEVARD PL NE,...,12/24/2020,18:26,603,501 BOULEVARD PL NE,23H,LARCENY-NON VEHICLE,Old Fourth Ward,M,33.769680,-84.370736
3,203651634,12/31/2020,12/30/2020,20:15,12/30/2020,22:25,211,NaN,NaN,3101 PIEDMONT RD NE,...,12/30/2020,22:25,211,3101 PIEDMONT RD NE,23F,LARCENY-FROM VEHICLE,Peachtree Park,B,33.840653,-84.369361
4,203660108,12/31/2020,12/31/2020,1:15,12/31/2020,1:15,306,NaN,NaN,1974 SYLVAN RD SW,...,12/31/2020,01:15,306,1974 SYLVAN RD SW,13A,AGG ASSAULT,Sylvan Hills,X,33.700432,-84.418128


In [9]:
# MERGE WITH CLEANED DATAFRAME
clean_merged202021_df = pd.merge(clean_merge2020_df, csvpath21_df, how="left", on="offense_id")
clean_merged202021_df.head()
new_clean_merge_df = len(clean_merged202021_df["offense_id"].unique())
print(f"There are {new_clean_merge_df} 'offense_ids' in the cleaned dataframe.")
clean_merged202021_df.head()

There are 7241 'offense_ids' in the cleaned dataframe.


,offense_id,rpt_date_x,occur_date_x,occur_time_x,poss_date_x,poss_time_x,beat_x,apt_office_prefix,apt_office_num,location_x,...,poss_date,poss_time,beat,location,ibr_code_y,UC2_Literal,neighborhood,npu,lat,long
0,203341346,12/31/2020,11/29/2020,19:21,11/29/2020,23:50,405,NaN,NaN,3050 M.L.K. JR DR SW,...,11/29/2020,23:50,405,3050 M.L.K. JR DR SW,13A,AGG ASSAULT,Harland Terrace,I,33.751060,-84.486163
1,203631200,12/31/2020,12/28/2020,17:21,12/28/2020,17:45,302,NaN,NaN,1186 IRA ST SW,...,12/28/2020,17:45,302,1186 IRA ST SW,220,BURGLARY,Pittsburgh,V,33.722696,-84.399763
2,203631285,12/31/2020,12/24/2020,14:00,12/24/2020,18:26,603,NaN,NaN,501 BOULEVARD PL NE,...,12/24/2020,18:26,603,501 BOULEVARD PL NE,23H,LARCENY-NON VEHICLE,Old Fourth Ward,M,33.769680,-84.370736
3,203651634,12/31/2020,12/30/2020,20:15,12/30/2020,22:25,211,NaN,NaN,3101 PIEDMONT RD NE,...,12/30/2020,22:25,211,3101 PIEDMONT RD NE,23F,LARCENY-FROM VEHICLE,Peachtree Park,B,33.840653,-84.369361
4,203660108,12/31/2020,12/31/2020,1:15,12/31/2020,1:15,306,NaN,NaN,1974 SYLVAN RD SW,...,12/31/2020,01:15,306,1974 SYLVAN RD SW,13A,AGG ASSAULT,Sylvan Hills,X,33.700432,-84.418128


In [10]:
# Renamed Headers to make it easir to read
clean_merged202021_df.rename(columns = {
    "offense_id":"Offense Id",
    "rpt_date_x":"Report Date",
    "occur_date_x":"Occurence Date",
    "occur_time_x":"Occurence Time",
    "poss_date_x":"Possession Date",
    "poss_time_x":"Possession Time",
    "beat_x":"Beat",
    "apt_office_prefix":"Apt Office Prefix",
    "apt_office_num":"Apt Office Num",
    "location_x":"Location",
    "poss_date":"Possession Date",
    "poss_time":"Possession Time",
    "beat":"Beat",
    "location":"Location",
    "ibr_code_y":"IBR code",
    "UC2_Literal":"UC2 Literal",
    "neighborhood":"Neighborhood",
    "npu":"NPU",
    "lat":"Latitude",
    "long":"Longitude"
}, inplace=True)
clean_merged202021_df.head()

,Offense Id,Report Date,Occurence Date,Occurence Time,Possession Date,Possession Time,Beat,Apt Office Prefix,Apt Office Num,Location,...,Possession Date,Possession Time,Beat,Location,IBR code,UC2 Literal,Neighborhood,NPU,Latitude,Longitude
0,203341346,12/31/2020,11/29/2020,19:21,11/29/2020,23:50,405,NaN,NaN,3050 M.L.K. JR DR SW,...,11/29/2020,23:50,405,3050 M.L.K. JR DR SW,13A,AGG ASSAULT,Harland Terrace,I,33.751060,-84.486163
1,203631200,12/31/2020,12/28/2020,17:21,12/28/2020,17:45,302,NaN,NaN,1186 IRA ST SW,...,12/28/2020,17:45,302,1186 IRA ST SW,220,BURGLARY,Pittsburgh,V,33.722696,-84.399763
2,203631285,12/31/2020,12/24/2020,14:00,12/24/2020,18:26,603,NaN,NaN,501 BOULEVARD PL NE,...,12/24/2020,18:26,603,501 BOULEVARD PL NE,23H,LARCENY-NON VEHICLE,Old Fourth Ward,M,33.769680,-84.370736
3,203651634,12/31/2020,12/30/2020,20:15,12/30/2020,22:25,211,NaN,NaN,3101 PIEDMONT RD NE,...,12/30/2020,22:25,211,3101 PIEDMONT RD NE,23F,LARCENY-FROM VEHICLE,Peachtree Park,B,33.840653,-84.369361
4,203660108,12/31/2020,12/31/2020,1:15,12/31/2020,1:15,306,NaN,NaN,1974 SYLVAN RD SW,...,12/31/2020,01:15,306,1974 SYLVAN RD SW,13A,AGG ASSAULT,Sylvan Hills,X,33.700432,-84.418128


In [19]:
# Cleaned merged dataframe with dropped off headers
crime_data = clean_merged202021_df.loc[:,["Offense Id","Report Date","Occurence Date","Occurence Time","Possession Time","Beat","IBR code","UC2 Literal","Neighborhood","NPU","Latitude","Longitude"]]
crime_data.head()

,Offense Id,Report Date,Occurence Date,Occurence Time,Possession Time,Possession Time,Beat,Beat,IBR code,UC2 Literal,Neighborhood,NPU,Latitude,Longitude
0,203341346,12/31/2020,11/29/2020,19:21,23:50,23:50,405,405,13A,AGG ASSAULT,Harland Terrace,I,33.751060,-84.486163
1,203631200,12/31/2020,12/28/2020,17:21,17:45,17:45,302,302,220,BURGLARY,Pittsburgh,V,33.722696,-84.399763
2,203631285,12/31/2020,12/24/2020,14:00,18:26,18:26,603,603,23H,LARCENY-NON VEHICLE,Old Fourth Ward,M,33.769680,-84.370736
3,203651634,12/31/2020,12/30/2020,20:15,22:25,22:25,211,211,23F,LARCENY-FROM VEHICLE,Peachtree Park,B,33.840653,-84.369361
4,203660108,12/31/2020,12/31/2020,1:15,1:15,01:15,306,306,13A,AGG ASSAULT,Sylvan Hills,X,33.700432,-84.418128


In [18]:
# # Cleaned merged dataframe with dropped off headers
cleaned_merged_df = crime_data.loc[:,~crime_data.columns.duplicated()]
cleaned_merged_df.head()

,Offense Id,Report Date,Occurence Date,Occurence Time,Possession Time,Beat,IBR code,UC2 Literal,Neighborhood,NPU,Latitude,Longitude
0,203341346,12/31/2020,11/29/2020,19:21,23:50,405,13A,AGG ASSAULT,Harland Terrace,I,33.751060,-84.486163
1,203631200,12/31/2020,12/28/2020,17:21,17:45,302,220,BURGLARY,Pittsburgh,V,33.722696,-84.399763
2,203631285,12/31/2020,12/24/2020,14:00,18:26,603,23H,LARCENY-NON VEHICLE,Old Fourth Ward,M,33.769680,-84.370736
3,203651634,12/31/2020,12/30/2020,20:15,22:25,211,23F,LARCENY-FROM VEHICLE,Peachtree Park,B,33.840653,-84.369361
4,203660108,12/31/2020,12/31/2020,1:15,1:15,306,13A,AGG ASSAULT,Sylvan Hills,X,33.700432,-84.418128


In [13]:
# # Cleaned merged dataframe with dropped off headers
# crime_data = clean_merged202021_df.loc[:,["Offense Id","Report Date","Occurence Date","Occurence Time","Possession Time","Beat","IBR code","UC2 Literal","Neighborhood","NPU","Latitude","Longitude"]]
# crime_data.head()

In [24]:
ibr_code_count = len(cleaned_merged_df["IBR code"].unique())
ibr_code_count
print(f"There are {ibr_code_count} unique IBR codes.")

There are 14 unique IBR codes.


In [25]:
uc2_literal_count = len(cleaned_merged_df["UC2 Literal"].unique())
uc2_literal_count
print(f"There are {uc2_literal_count} unique UC2 literal counts.")

There are 8 unique UC2 literal counts.


In [26]:
neighborhood_count = len(cleaned_merged_df["Neighborhood"].unique())
neighborhood_count
print(f"There are {neighborhood_count} neighborhoods.")

There are 203 neighborhoods.


In [27]:
npu_count = len(cleaned_merged_df["NPU"].unique())
npu_count
print(f"There are {npu_count} NPU's.cleaned_merged_df")

There are 26 NPU's.cleaned_merged_df
